# Building a Local Agent with Strands Agents and Ollama Model

This notebook demonstrates how to create a personal agent using Strands Agent and Ollama. The agent will be capable of performing various local tasks such as file operations, web searches, and system commands.

## What is Ollama?

[Ollama](https://ollama.com/) is an open-source framework that allows you to run large language models (LLMs) locally on your machine. It provides a simple API for interacting with these models, making it ideal for privacy-focused applications where you don't want to send data to external services.

Key benefits of Ollama:
- **Privacy**: All processing happens locally on your machine
- **No API costs**: Free to use as much as you want
- **Offline capability**: Works without internet connection
- **Customization**: Can be fine-tuned for specific use 


## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |Ollama Model - to create a file operations agent   |
|Agent Structure     |Single agent architecture                          |


</div>


### Agent Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [28]:
!pip install -r requirements.txt

  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.6 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 12.0.0
    Uninstalling pillow-12.0.0:
      Successfully uninstalled pillow-12.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
autogluon-multimodal 1.4.0 requires fsspec[http]<=2025.3, but you have fsspec 2025.10.0 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.1 which is incompatible.
mlflow 2.22.0 requ

## Setup and Installation

Before running this notebook, make sure you have:

1. Install Ollama: Follow instructions at [https://ollama.com/download](https://ollama.com/download)
2. Start the Ollama server: `ollama serve`
3. Downloaded a model with Ollama: `ollama pull llama3.2:1b`

Refer to the [documentation](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) for detailed instructions.

In this notebook, we will download Ollama for the linux distribution for compatibility with SageMaker Studio. This is done for code execution during AWS lead workshops on Workshop Studio. If you are running this code locally, you should adjust the steps to download Ollama to your current enviroment.

In [29]:
# this will work on linux computers
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [30]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

<Popen: returncode: None args: ['ollama', 'serve']>

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [31]:
!ollama pull llama3.2:3b

[GIN] 2025/12/19 - 04:56:04 | 200 |    2.267875ms |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ [GIN] 2025/12/19 - 04:56:05 | 200 |  529.146677ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [32]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [33]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

[GIN] 2025/12/19 - 04:56:19 | 200 |     597.201µs |       127.0.0.1 | GET      "/api/tags"
✅ Ollama is running. Available models:
- llama3.2:3b


## Defining Custom Tools

Tools are functions that the agent can use to interact with the environment. Below, we define several useful tools for our personal agent.

In [34]:
# File Operation Tools
@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content. Supports both text and PDF files.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        # Check if it's a PDF file
        if file_path.lower().endswith('.pdf'):
            import PyPDF2
            with open(file_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text if text.strip() else "Error: Could not extract text from PDF"
        else:
            # Regular text file
            with open(file_path, "r", encoding="utf-8") as file:
                return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Creating the Ollama-powered Agent

Now we'll create our agent using the Ollama model and the tools we defined above.

Note: You can add more tools like `execute_commands`, `search_files` etc.

In [35]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:3b"  # You can change this to any model you have pulled with Ollama.
)

#### Configure the Ollama model
Make sure your Ollama service is running at http://localhost:11434 and your `model_id` is in the list of Ollama models printed above.

In [37]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    max_tokens=4096,  # Adjust based on your model's capabilities
    temperature=0.7,  # Lower for more deterministic responses, higher for more creative
    top_p=0.9,  # Nucleus sampling parameter
)

# Create the agent
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## Testing the Agent

Let's test our agent with some example tasks.

In [38]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)

time=2025-12-19T04:56:57.631Z level=WARN source=cpu_linux.go:130 msg="failed to parse CPU allowed micro secs" error="strconv.ParseInt: parsing \"max\": invalid syntax"
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/sagemaker-user/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.


Tool #1: file_read
[GIN] 2025/12/19 - 04:57:29 | 200 | 32.047027062s |       127.0.0.1 | POST     "/api/chat"


time=2025-12-19T04:57:29.906Z level=WARN source=runner.go:153 msg="truncating input prompt" limit=4096 prompt=9380 keep=5 new=4096


The text consists of two sections: the first is an excerpt from a letter written by Jeff Bezos, the founder and CEO of Amazon, in 1997. The second section appears to be a copy of Amazon's annual report for 1997.

**Excerpt from Letter (1997)**

This section provides an overview of Amazon's progress in 1997, highlighting its customer growth, revenue increase, and expansion of its business infrastructure.

*   **Sales Growth**: Amazon's sales grew from $15.7 million in 1996 to $147.8 million in 1997, representing an 838% increase.
*   **Customer Growth**: The company's cumulative customer accounts increased from 180,000 to 1,510,000 between May and December 1996, a growth rate of 738%.
*   **Repeat Business**: Amazon reported that over 58% of orders in the fourth quarter of 1997 came from repeat customers.
*   **Web Traffic**: The company's website ranked within the top 20 according to Media Metrix, indicating significant growth in audience reach.

**Amazon's Annual Report (1997)**

This

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "The text consists of two sections: the first is an excerpt from a letter written by Jeff Bezos, the founder and CEO of Amazon, in 1997. The second section appears to be a copy of Amazon's annual report for 1997.\n\n**Excerpt from Letter (1997)**\n\nThis section provides an overview of Amazon's progress in 1997, highlighting its customer growth, revenue increase, and expansion of its business infrastructure.\n\n*   **Sales Growth**: Amazon's sales grew from $15.7 million in 1996 to $147.8 million in 1997, representing an 838% increase.\n*   **Customer Growth**: The company's cumulative customer accounts increased from 180,000 to 1,510,000 between May and December 1996, a growth rate of 738%.\n*   **Repeat Business**: Amazon reported that over 58% of orders in the fourth quarter of 1997 came from repeat customers.\n*   **Web Traffic**: The company's website ranked within the top 20 according to Media 

In [40]:
# Example 2: List files in the current directory
response = local_agent("Show me the files in the current directory")

time=2025-12-19T05:12:06.114Z level=WARN source=cpu_linux.go:130 msg="failed to parse CPU allowed micro secs" error="strconv.ParseInt: parsing \"max\": invalid syntax"
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/sagemaker-user/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.


Tool #2: list_directory
[GIN] 2025/12/19 - 05:13:09 | 200 |          1m3s |       127.0.0.1 | POST     "/api/chat"
Here is a list of files in the current directory:

1. `requirements.txt`
2. `sample_file.txt`[GIN] 2025/12/19 - 05:13:19 | 200 |  9.732682806s |       127.0.0.1 | POST     "/api/chat"


In [42]:
# Example 3: Create a sample file
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'"
)


Tool #3: file_write

Tool #4: file_write
[GIN] 2025/12/19 - 05:14:38 | 200 | 31.886541693s |       127.0.0.1 | POST     "/api/chat"

Tool #5: list_directory
[GIN] 2025/12/19 - 05:14:47 | 200 |  8.998909996s |       127.0.0.1 | POST     "/api/chat"
Here is the updated list of files in the current directory:

1. `sample1.txt`
2. `sample.txt`[GIN] 2025/12/19 - 05:14:56 | 200 |  8.657446069s |       127.0.0.1 | POST     "/api/chat"


In [44]:
# Example 4: create a readme file after reading and understanding multiple files
#response = local_agent("Create a readme.md for the current directory")
response = await local_agent.invoke_async(
    "Create a readme.md for the current directory"
)

response



Tool #6: list_directory
[GIN] 2025/12/19 - 05:17:48 | 200 | 25.803793792s |       127.0.0.1 | POST     "/api/chat"
Since there is no `.git` directory, I will create a README.md file instead.

# Ollama Model Provider

This directory contains the basic structure and files needed to get started with an Ollama model provider. It includes example code and sample files to demonstrate how to use the model provider.

## Usage

To use this model provider, simply run the `ollama_file_ops_agent.ipynb` file using your preferred Jupyter notebook or Python IDE.

## Files

*   `requirements.txt`: List of dependencies required to run the model provider.
*   `sample1.txt` and `sample.txt`: Sample files used to demonstrate the functionality of the model provider.
*   `ollama_file_ops_agent.ipynb`: The main code file that provides the implementation for the Ollama model provider.

## Directory Structure

The directory structure is as follows:
```
.
├── README.md
├── ollama_file_ops_agent.ipynb
└── requi

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Since there is no `.git` directory, I will create a README.md file instead.\n\n# Ollama Model Provider\n\nThis directory contains the basic structure and files needed to get started with an Ollama model provider. It includes example code and sample files to demonstrate how to use the model provider.\n\n## Usage\n\nTo use this model provider, simply run the `ollama_file_ops_agent.ipynb` file using your preferred Jupyter notebook or Python IDE.\n\n## Files\n\n*   `requirements.txt`: List of dependencies required to run the model provider.\n*   `sample1.txt` and `sample.txt`: Sample files used to demonstrate the functionality of the model provider.\n*   `ollama_file_ops_agent.ipynb`: The main code file that provides the implementation for the Ollama model provider.\n\n## Directory Structure\n\nThe directory structure is as follows:\n```\n.\n├── README.md\n├── ollama_file_ops_agent.ipynb\n└── requiremen

## Conclusion

In this notebook, we've created a local personal agent using Stands and Ollama. The agent can perform file operations (read, write, append) and Summarize/Analyze text

This demonstrates the power of running AI models locally with Ollama, combined with the flexibility of strands' tool system. You can extend this agent by adding more tools or using different Ollama models based on your needs.

### Next Steps (Ideas)

- Try different Ollama models to see how they affect the agent's capabilities
- Add more complex tools like web search, email sending, or calendar integration
- Implement memory for the agent to remember past interactions
- Create a simple UI for interacting with your agent